In [8]:
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4, landscape
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.units import inch

# ------------------------------
# Load Excel
# ------------------------------
df = pd.read_excel("Library Cleaned.xlsx")

# Define mapping of subjects to keywords
subject_keywords = {
    "Business Ethics and Human Values": ["ethics", "values"],
    "Business Application Tools - III": ["application", "tools"],
    "Blockchain and Cryptocurrencies": ["blockchain", "crypto"],
    "Digital Payments and Banking": ["digital payment", "banking"],
    "Behavioral Finance in the Digital Era": ["finance", "behavioral"],
    "Peer-To-Peer Lending and Crowd Funding": ["lending", "crowd funding"],
    "Python for Data Science": ["python", "data science"],
    "Financial Analytics": ["finance", "financial", "accounting"],
    "Big Data Technologies": ["big data", "data"],
    "Data Visualisation": ["visualisation", "research methods"],
    "Services Marketing": ["services marketing"],
    "Integrated Marketing Communication": ["communication", "marketing"],
    "Retail Management": ["retail", "management"],
    "Consumer Behaviour": ["consumer", "behaviour", "behavior"],
    "Regulatory Framework in Banking and Finance": ["regulatory", "banking", "finance"],
    "Investment Analysis and Portfolio Management": ["investment", "portfolio"],
    "Labour Relations and Employment Law": ["labour", "employment", "law"],
    "Organizational Development and Change Management": ["organizational", "development", "change"],
    "Inventory Management": ["inventory"],
    "Logistics Management": ["logistics"]
}

# ------------------------------
# Step 1: Arrange books into Wings and Shelves
# ------------------------------
def arrange_books(df, max_books_per_shelf=10):
    """Distribute books across shelves in wings"""
    wings = ["A", "B", "C"]

    df = df.reset_index(drop=True)
    df["Shelf_No"] = (df.index // max_books_per_shelf) + 1
    df["Wing"] = df["Shelf_No"].apply(lambda x: wings[(x - 1) // 30])

    return df

df = arrange_books(df)

# ------------------------------
# Step 2: Search function
# ------------------------------
def search_books(subject):
    """Search for books related to a subject based on keywords"""
    keywords = subject_keywords.get(subject, [])
    if not keywords:
        return None, f"No keywords mapped for subject: {subject}"

    mask = df['Title'].str.lower().str.contains('|'.join(keywords), case=False)
    result = df[mask]

    if result.empty:
        return None, f"No books found for subject: {subject}"
    else:
        return result[["Title", "Author", "Publication", "Qty", "Wing", "Shelf_No"]], None

# ------------------------------
# Step 3: Export to PDF (Professional Report Style)
# ------------------------------
def export_to_pdf(dataframe, subject, filename="output.pdf"):
    """Export search results to PDF in a professional, well-formatted style"""

    # Use landscape for wide tables
    page_size = landscape(A4)
    doc = SimpleDocTemplate(filename, pagesize=page_size,
                            rightMargin=20, leftMargin=20, topMargin=20, bottomMargin=20)
    elements = []

    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name="TableCell", fontSize=8, leading=10, alignment=0))  # left align text
    styles.add(ParagraphStyle(name="TableCellCenter", fontSize=8, leading=10, alignment=1))  # center align

    # Title
    title = Paragraph(f"<b>Books for Subject: {subject}</b>", styles['Title'])
    elements.append(title)
    elements.append(Spacer(1, 12))

    # Prepare data with wrapped text
    data = []
    headers = list(dataframe.columns)
    data.append(headers)

    for _, row in dataframe.iterrows():
        row_data = []
        for col in headers:
            text = str(row[col])
            if col in ["Title", "Author", "Publisher"]:
                row_data.append(Paragraph(text, styles["TableCell"]))
            else:
                row_data.append(Paragraph(text, styles["TableCellCenter"]))
        data.append(row_data)

    # Custom column widths
    col_widths = [
        3.5 * inch,  # Title
        1.8 * inch,  # Author
        1.5 * inch,  # Publisher
        0.6 * inch,  # Qty
        0.6 * inch,  # Wing
        0.8 * inch,  # Shelf
    ]

    table = Table(data, colWidths=col_widths, repeatRows=1)

    # Styling
    table.setStyle(TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.HexColor("#1F4E79")),  # dark blue header
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
        ("ALIGN", (0, 0), (-1, 0), "CENTER"),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("FONTSIZE", (0, 0), (-1, 0), 9),
        ("BOTTOMPADDING", (0, 0), (-1, 0), 8),

        ("GRID", (0, 0), (-1, -1), 0.25, colors.black),
        ("ROWBACKGROUNDS", (0, 1), (-1, -1), [colors.whitesmoke, colors.lightgrey]),

        ("VALIGN", (0, 0), (-1, -1), "TOP"),  # align text to top
    ]))

    elements.append(table)
    doc.build(elements)
    print(f"✅ Professional PDF created: {filename}")
# ------------------------------
# Example usage
# ------------------------------
subject = "Integrated Marketing Communication"
result, msg = search_books(subject)

if msg:  # if there was an error or no results
    print(msg)
else:
    print(result)  # show in console
    export_to_pdf(result, subject, filename="IMC_books.pdf")


                                                 Title           Author  \
17                                Marketing Management    Kotler Philip   
18                                Marketing Management     Saxena Rajan   
19                             Principles Of Marketing    Kotler Philip   
20                                Marketing Management    Kotler Philip   
37   The Arts & Science Of Business Communication S...   Chaturvedi P.D   
45                        Business Communication Today  Bovee Churtland   
52                             Principles Of Marketing    Kotler Philip   
79                          Communication For Business   Tayler Shiniey   
83                                   Digital Marketing      Gupta Seema   
84                                  Services Marketing    Verma V Harsh   
85                            Rural Marketing In India       Habeeb K.S   
101                                 Marketing Research       Luck David   
126                      